# Домашнее задание 2

Используя датасет [STUDENT ALCOHOL CONSUMPTION](https://archive.ics.uci.edu/ml/datasets/STUDENT+ALCOHOL+CONSUMPTION#) и статью "USING	DATA	MINING	TO	PREDICT	SECONDARY SCHOOL	STUDENT	ALCOHOL	CONSUMPTION" выполните следующие задания:

### Предобработка (Max 2)
- Приведите описание признаков датасета на русском языке с указанием типа данных (0.3 балла)
- Предобработка данных (1.7)
    - Проверьте наличие пропусков. В случае наличия пропусков заполните их медианными значениями (0.4)
    - Подсчитайте количество людей, у которых матери с образованием "0" (0.4)
    - Преобразуйте все номинальные признаки в несколько признаков с бинарным значение (0.4)
    - Используя объект [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest) с параметром [f_regression](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)  выберите два признака, которые наиболее сильно связаны с признаком G3 (0.5)

In [164]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error,r2_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import RidgeCV, SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
%matplotlib inline

In [110]:
df = pd.read_csv("student-por.csv",sep=';')

In [111]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


## Проверьте наличие пропусков. В случае наличия пропусков заполните их медианными значениями 

In [112]:
True in pd.isnull(df)

False

### В данных нет пропусков

## Подсчитайте количество людей, у которых матери с образованием "0" 

In [113]:
sum(df['Medu']==0)

6

### Есть 6 людей, у которых матери с образованием "0"

## Преобразуйте все номинальные признаки в несколько признаков с бинарным значением

In [114]:
df = pd.get_dummies(df, columns = ['Mjob', 'Fjob', 'reason', 'guardian'])

In [115]:
df.columns.values

array(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu',
       'Fedu', 'traveltime', 'studytime', 'failures', 'schoolsup',
       'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet',
       'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',
       'absences', 'G1', 'G2', 'G3', 'Mjob_at_home', 'Mjob_health',
       'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_course', 'reason_home', 'reason_other', 'reason_reputation',
       'guardian_father', 'guardian_mother', 'guardian_other'], dtype=object)

### Так же преобразуем бинарные признаки закодированные словами в бинарные закодированные числами
### (SelectKBest не будет работать со строковыми значениями)

In [116]:
def binarize(df, cols):
    for name in cols:
        lb = LabelBinarizer()
        df[name] = lb.fit_transform(df[name])

In [117]:
binarize(df,['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid',
      'activities', 'nursery', 'higher', 'internet', 'romantic'])

In [118]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0


## Используя объект SelectKBest с параметром f_regression выберите два признака, которые наиболее сильно связаны с признаком G3

In [119]:
kb = SelectKBest(score_func=f_regression,k=2)

In [121]:
best_feat = kb.fit_transform(df.drop("G3",axis=1),df["G3"])

In [122]:
kb.get_support(indices=True)

array([26, 27], dtype=int64)

In [123]:
df.drop("G3",axis=1).columns.values[26:28]

array(['G1', 'G2'], dtype=object)

### Регрессия (Max 4)
Выберите один признак, который наиболее влияет на признак G3 и используя кросс-валидацию (cross-validation) выполните следующее:

In [124]:
kb = SelectKBest(score_func=f_regression,k=1)
best_feat = kb.fit_transform(df.drop("G3",axis=1),df["G3"])
kb.get_support(indices=True)

array([27], dtype=int64)

In [126]:
df.drop("G3",axis=1).columns.values[27]

'G2'

- На обучающей выборке обучите объект [RidgeСV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV) для различных alpha (0.2)
- Выберите alpha при котором RidgeCV лучше всего предсказывает значения (0.1)
- Подсчитайте метрику mean absolute error, mean squared error, median absolute error,r2 score(0.2)
- Max (0.5)
- ------------------------------------
- Создайте объекты [SGDRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor), используя все возможных варианты аргумента penalty (0.2)
- Обучите все полученных объекты на обучающей выборке (0.1)
- Подсчитайте метрику r2 score для всех объектов (0.2)
- Max (0.5)
- ------------------------------------
- Создайте объекты с параметрами max_depth = 1,3, 10 [GradientBoostingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor) и для каждого объекта изменяйте количество n_estimators  от 100 до 1000 с шагом 100 (0.2)
- Подсчитайте метрику r2 score для всех объектов (0.1)
- Визуализируйте зависимость R2 oт n_estimators на графике (0.2)
- Max (0.5)
- -----------------------------------
- Используя все доступные регрессоры в библиотеке sklearn, найдите самый лучший регрессор. (2.5)

# RidgeCV

In [130]:
X = np.array(df['G2']).reshape(-1,1)
y = np.array(df['G3'])

In [141]:
rcv = RidgeCV(alphas=np.logspace(-3,2,20))

In [142]:
rcv.fit(X,y)

RidgeCV(alphas=array([  1.00000e-03,   1.83298e-03,   3.35982e-03,   6.15848e-03,
         1.12884e-02,   2.06914e-02,   3.79269e-02,   6.95193e-02,
         1.27427e-01,   2.33572e-01,   4.28133e-01,   7.84760e-01,
         1.43845e+00,   2.63665e+00,   4.83293e+00,   8.85867e+00,
         1.62378e+01,   2.97635e+01,   5.45559e+01,   1.00000e+02]),
    cv=None, fit_intercept=True, gcv_mode=None, normalize=False,
    scoring=None, store_cv_values=False)

In [143]:
rcv.alpha_

2.6366508987303581

### Лучшее alpha = 2.6366508987303581

In [146]:
y_pred = rcv.predict(X)

In [147]:
mean_absolute_error(y_pred,y)

0.8078914385784195

In [148]:
mean_squared_error(y_pred,y)

1.6284962012799604

In [149]:
median_absolute_error(y_pred,y)

0.62035468509676761

In [150]:
r2_score(y_pred,y)

0.81460951505566415

# SGDRegressor

In [152]:
sgdr_l1 = SGDRegressor(penalty='l1')
sgdr_l2 = SGDRegressor(penalty='l2')
sgdr_eln = SGDRegressor(penalty='elasticnet')

C:\Users\Alexander\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [154]:
sgdr_l1.fit(X,y);
sgdr_l2.fit(X,y);
sgdr_eln.fit(X,y);

In [155]:
y_pred_l1 = sgdr_l1.predict(X)
y_pred_l2 = sgdr_l2.predict(X)
y_pred_eln = sgdr_eln.predict(X)

In [157]:
r2_score(y_pred_l1,y)

0.81256470147788995

In [158]:
r2_score(y_pred_l2,y)

0.8156906321352474

In [159]:
r2_score(y_pred_eln,y)

0.76332410246170057

# GradientBoostingRegressor

In [165]:
md = [1,3,10]
ne = np.arange(100,1100,100)
results = pd.DataFrame()

In [166]:
for max_d in md:
    for n_est in ne:
        gbr = GradientBoostingRegressor(max_depth=max_d,n_estimators=n_est)
        gbr.fit(X,y)
        y_pred = gbr.predict(X)
        score = r2_score(y_pred,y)
        results = results.append({'Parameters':{'max_depth':max_d,'n_estimators':n_est},'R2 score':score},ignore_index=True)

In [167]:
results

,Parameters,R2 score
0,"{'max_depth': 1, 'n_estimators': 100}",0.817225
1,"{'max_depth': 1, 'n_estimators': 200}",0.825700
2,"{'max_depth': 1, 'n_estimators': 300}",0.826037
3,"{'max_depth': 1, 'n_estimators': 400}",0.826087
4,"{'max_depth': 1, 'n_estimators': 500}",0.826111
5,"{'max_depth': 1, 'n_estimators': 600}",0.826125
6,"{'max_depth': 1, 'n_estimators': 700}",0.826132
7,"{'max_depth': 1, 'n_estimators': 800}",0.826137
8,"{'max_depth': 1, 'n_estimators': 900}",0.826139
9,"{'max_depth': 1, 'n_estimators': 1000}",0.826140


In [168]:
results['R2 score'].idxmax()

13

### Классификация (Max 4)
- В статье описано использование Random Forest для предсказания важности факторов, влияющих на потребление алкоголя. Повторите эксперимент с использование объекта [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) и найдите наилучшие параметры. Укажите accuracy для наилучших параметров (0.5)
- Обучите объект [GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)  и найдите для него наилучшие параметры. Укажите accuracy для наилучших параметров (0.5)
- Обучите объект [AdaBoostClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)  и найдите для него наилучшие параметры. Укажите accuracy для наилучших параметров (0.5)
- Найдите наилучший классификатор. Укажите accuracy для наилучших параметров (2.5)

# Замечания

- В работе следует использовать библиотеку scikit-learn версии 0.18 и scipy версии 0.18.1